## Step 1: Install and Import Packages

We begin by installing and importing the essential libraries:  
- **PyTorch:** for building and training the Autoencoder model  
- **Scikit-learn:** for preprocessing (scaling features)  
- **Pandas & Numpy:** for dataset handling  
- **Matplotlib/Seaborn:** for visualisation  

These tools provide the foundation for constructing and evaluating our anomaly detection pipeline.


In [1]:
# pip install pandas 
# pip install 
# pip install numpy 
# pip install scikit-learn 
# pip install scipy 
# pip install torch

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

## Step 2: Loading the Dataset

Next, we load the **pre-processed review dataset** (`final_dataset.parquet`).  
This dataset contains engineered features derived from the raw Google reviews, such as:  
- Text embeddings (capturing semantic meaning)  
- Reviewer metadata (e.g., review frequency, business overlap)  

These features serve as input to the Autoencoder model for anomaly detection.


In [3]:
file_path = 'final_dataset.parquet'

if not os.path.exists(file_path):
	display(f"Error: The file '{file_path}' was not found. Please ensure it's in the correct directory.")
	exit() 
else:
	try:
		df = pd.read_parquet(file_path)
		display(f"Successfully loaded '{file_path}'. Shape: {df.shape}")
		print(f"Number of rows (lines): {df.shape[0]}")
		display("First 5 rows of the dataset:")
		display(df.head())
	except Exception as e:
		display(f"Error loading Parquet file: {e}")
		exit() 

"Successfully loaded 'final_dataset.parquet'. Shape: (26622, 36)"

Number of rows (lines): 26622


'First 5 rows of the dataset:'

,user_id,name_review_user,time,rating,text,gmap_id,latitude,longitude,category,avg_rating,...,year,month,weekday,hour,text_len,log_num_reviews,rating_deviation,sentiment_polarity,sentiment_subjectivity,excessive_exclaim
0,107813727678662505612,Felicia Wall,1575939675467,5,Coffee is great and so is the service,0x56c67c7da439c267:0x72f12aacd98f7f2e,60.561418,-151.285291,[Cafe],4.6,...,2019,12,1,1,37,4.290459,0.4,0.800000,0.750000,False
1,103473424444871906765,Garry Hanley,1602702499311,5,Easy in and out experience. Reasonable pricin...,0x56c899de8ee5d2a5:0xb15f3c48a1dc94f0,61.137792,-149.836917,"[Department of motor vehicles, Auto tag agency...",4.2,...,2020,10,2,19,109,4.744932,0.8,0.377778,0.644444,False
2,104393976961042472712,Nathan Harrison,1559438084515,4,"Great lunch, service could be better as there ...",0x56c894263c293e31:0x39aa7cb841273597,61.228926,-149.741000,"[Restaurant, Brewpub, Pizza restaurant, Sports...",3.9,...,2019,6,6,1,71,6.475433,0.1,0.600000,0.583333,False
3,110829207026204377440,Terry Oakes,1578766259352,3,Small portions and do not give you what the me...,0x51325b0542590ed1:0xaba794ec633521ef,64.821774,-147.713617,[Restaurant],4.2,...,2020,1,5,18,55,4.836282,-1.2,-0.250000,0.400000,False
4,108158723955787868825,Lee Petersen,1501725448031,4,"Nice gym, what I expect from a Planet Fitness....",0x51325acaa661fa4d:0xa0cf5a8e5d34cbde,64.835650,-147.742050,"[Gym, Fitness center, Personal trainer, Spa an...",4.1,...,2017,8,3,1,152,5.342334,-0.1,0.241667,0.741667,False


## Step 3: Data Preprocessing

Before feeding the data into the model, we perform preprocessing:  

1. **Feature Selection:** Extract the relevant feature columns.  
2. **Scaling:** Apply `MinMaxScaler` to normalise values into the **[0,1] range**.  
   - Ensures fair contribution of each feature.  
3. **Conversion to PyTorch Tensors:** Required format for PyTorch models.  
4. **DataLoader Creation:** Allows efficient **mini-batch training**, shuffling, and GPU compatibility.  

This step ensures our Autoencoder can efficiently learn meaningful representations from the dataset.


In [4]:
# Feature Selection 
features = [
	'rating',
	'text_len',
	'rating_deviation',
	'sentiment_polarity',
	'sentiment_subjectivity',
	'excessive_exclaim',
	'avg_rating',
	#'num_of_reviews',
	'log_num_reviews',
	# 'latitude',
	# 'longitude',
	'price_encoded',
	'year',
	'month',
	'weekday',
	'hour',
	'cat_American restaurant',
	'cat_Coffee shop',
	'cat_Department store',
	'cat_Fast food restaurant',
	'cat_Grocery store',
	'cat_Hotel',
	'cat_Mexican restaurant',
	'cat_Other',
	'cat_Pizza restaurant',
	'cat_Restaurant',
	'cat_Shopping mall'
]

# Check if all required features exist in the DataFrame
missing_features = [f for f in features if f not in df.columns]
if missing_features:
	display(f"Error: Missing required features in the dataset: {missing_features}")
	display("Please ensure your 'final_dataset.parquet' contains these columns.")
	features = [f for f in features if f in df.columns] 
	if not features:
		display("No valid features remaining. Exiting.")
		exit()
	else:
		display(f"Proceeding with available features: {features}")

data = df[features].copy()


# Data Preprocessing (Scaling)
scaler = MinMaxScaler()
scaled_data_np = scaler.fit_transform(data) 

# Convert to PyTorch tensors
scaled_data_tensor = torch.tensor(scaled_data_np, dtype=torch.float32)

# Split data into training and validation sets
X_train_tensor, X_val_tensor = train_test_split(scaled_data_tensor, test_size=0.2, random_state=42)

# Create DataLoader for batching
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, X_train_tensor) 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## Step 4: Autoencoder Model Architecture (PyTorch)

We define the **Autoencoder** using PyTorch’s `nn.Module`.  

**Key Concepts:**  
- The Autoencoder learns to **reconstruct its input**.  
- Normal data → low reconstruction error.  
- Anomalies → high reconstruction error (poorly reconstructed).  

**Architecture Details:**  
- **Input layer:** 24 features from the dataset.  
- **Encoder:** Compresses input into a **10-dimensional latent space**.  
- **Decoder:** Reconstructs the original 24 features from the latent vector.  

By compressing and reconstructing, the model captures underlying patterns in the data.  
Large reconstruction errors suggest unusual (potentially spam) reviews.


In [5]:
# Autoencoder Model Definition (PyTorch)
class Autoencoder(nn.Module):
	def __init__(self, input_dim, encoding_dim):
		super(Autoencoder, self).__init__()
		self.encoder = nn.Sequential(
			nn.Linear(input_dim, encoding_dim * 2),
			nn.LeakyReLU(0.01),  
			nn.Linear(encoding_dim * 2, encoding_dim),
			nn.LeakyReLU(0.01)   
		)
		self.decoder = nn.Sequential(
			nn.Linear(encoding_dim, encoding_dim * 2),
			nn.LeakyReLU(0.01),  
			nn.Linear(encoding_dim * 2, input_dim),
			nn.Sigmoid()
		)
		
	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		return decoded

input_dim = X_train_tensor.shape[1]
# Dimensionality of the latent space
encoding_dim = 10 

model = Autoencoder(input_dim, encoding_dim)
display("\nAutoencoder Model Summary (PyTorch):")
display(model) 

# Define Loss function and Optimiser
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

'\nAutoencoder Model Summary (PyTorch):'

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=24, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=20, out_features=24, bias=True)
    (3): Sigmoid()
  )
)

## Step 5: Training the Autoencoder

We train the model using an **unsupervised learning approach**:  

1. **Forward Pass:** Input data → latent space → reconstructed output.  
2. **Loss Function:** Mean Squared Error (MSE) between input and output.  
3. **Backpropagation:** Update weights to minimise reconstruction error.  
4. **Validation Monitoring:** Use a held-out validation set to detect overfitting.  

Training continues for multiple **epochs** until the model stabilises, learning to reconstruct normal reviews effectively.



In [6]:
# Training the Autoencoder
epochs = 50
display(f"\nTraining Autoencoder for {epochs} epochs...")

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
display(f"Using device: {device}")

for epoch in range(epochs):
	model.train() 
	running_loss = 0.0
	for data_batch, _ in train_loader:
		data_batch = data_batch.to(device)

		optimizer.zero_grad() 
		outputs = model(data_batch) 
		loss = criterion(outputs, data_batch) 
		loss.backward() 
		optimizer.step() 

		running_loss += loss.item() * data_batch.size(0)

	train_loss = running_loss / len(train_dataset)

	# Validation phase
	model.eval() 
	val_loss = 0.0
	with torch.no_grad(): 
		for data_batch_val, _ in val_loader:
			data_batch_val = data_batch_val.to(device)
			outputs_val = model(data_batch_val)
			loss_val = criterion(outputs_val, data_batch_val)
			val_loss += loss_val.item() * data_batch_val.size(0)
	val_loss = val_loss / len(val_dataset)

	display(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

display("\nTraining complete.")

'\nTraining Autoencoder for 50 epochs...'

'Using device: cpu'

'Epoch 1/50, Train Loss: 0.0541, Val Loss: 0.0360'

'Epoch 2/50, Train Loss: 0.0283, Val Loss: 0.0233'

'Epoch 3/50, Train Loss: 0.0222, Val Loss: 0.0199'

...

'Epoch 47/50, Train Loss: 0.0020, Val Loss: 0.0019'

'Epoch 48/50, Train Loss: 0.0019, Val Loss: 0.0018'

'Epoch 49/50, Train Loss: 0.0019, Val Loss: 0.0019'

'Epoch 50/50, Train Loss: 0.0019, Val Loss: 0.0019'

'\nTraining complete.'

## Step 6: Anomaly Detection via Reconstruction Error

After training, we use the Autoencoder to detect anomalies:  

- For each review, compute **Reconstruction Error = ||x - x'||²**.  
- Reviews with **high errors** are flagged as anomalies.  

This works because:  
- Normal reviews → follow learned patterns → low error.  
- Spam reviews → deviate from patterns → high error.  

We then export these anomaly scores for further fusion with the Isolation Forest results.


In [7]:
# Calculate Reconstruction Error (PyTorch) 
model.eval() 
all_reconstructions = []
with torch.no_grad():
	for i in range(0, len(scaled_data_tensor), batch_size):
		batch = scaled_data_tensor[i:i + batch_size].to(device)
		reconstructed_batch = model(batch)
		all_reconstructions.append(reconstructed_batch.cpu().numpy())

reconstructions_np = np.vstack(all_reconstructions)

# Calculate Mean Squared Error (MSE) between original and reconstructed data
reconstruction_errors = np.mean(np.square(scaled_data_np - reconstructions_np), axis=1)

# Z-scoring the Reconstruction Error
z_scored_reconstruction_errors = zscore(reconstruction_errors)

# Add the z-scored errors to the original dataframe
df['ae_reconstruction_error_zscore'] = z_scored_reconstruction_errors

display("\nReconstruction errors calculated and z-scored.")
display("First 10 reviews with their z-scored autoencoder reconstruction error:")
display(df[['ae_reconstruction_error_zscore'] + features].head(10))

# Identify potential anomalies
threshold = 2.0

# Add binary "is_outlier" column (1 if anomaly, 0 if normal)
df['is_outlier'] = np.where(
    df['ae_reconstruction_error_zscore'] > threshold,
    -1,   # outlier
    1     # normal
)
print(df['is_outlier'].value_counts())

anomalies = df[df['ae_reconstruction_error_zscore'] > threshold]
display(f"\nNumber of potential anomalies (z-score > {threshold}): {len(anomalies)}")

# Display more anomalies and all of their original features (For viewing purposes)
if not anomalies.empty:
	print("\nDisplaying the top 20 anomalies with all their features (sorted by anomaly score):")

	pd.set_option('display.max_rows', 50)       
	pd.set_option('display.max_columns', None)   
	pd.set_option('display.width', 1000)          
	pd.set_option('display.max_colwidth', 150)    

	# Sort by the anomaly score to see the most anomalous first and show the top 20
	print(anomalies.sort_values(by='ae_reconstruction_error_zscore', ascending=False).head(20))

else:
	print("\nNo anomalies detected above the specified threshold.")

'\nReconstruction errors calculated and z-scored.'

'First 10 reviews with their z-scored autoencoder reconstruction error:'

,ae_reconstruction_error_zscore,rating,text_len,rating_deviation,sentiment_polarity,sentiment_subjectivity,excessive_exclaim,avg_rating,log_num_reviews,price_encoded,...,cat_Coffee shop,cat_Department store,cat_Fast food restaurant,cat_Grocery store,cat_Hotel,cat_Mexican restaurant,cat_Other,cat_Pizza restaurant,cat_Restaurant,cat_Shopping mall
0,-0.645158,5,37,0.4,0.800000,0.750000,False,4.6,4.290459,0,...,False,False,False,False,False,False,True,False,False,False
1,-0.555159,5,109,0.8,0.377778,0.644444,False,4.2,4.744932,0,...,False,False,False,False,False,False,True,False,False,False
2,-0.546985,4,71,0.1,0.600000,0.583333,False,3.9,6.475433,2,...,False,False,False,False,False,False,False,False,True,False
3,1.402167,3,55,-1.2,-0.250000,0.400000,False,4.2,4.836282,0,...,False,False,False,False,False,False,False,False,True,False
4,-0.792028,4,152,-0.1,0.241667,0.741667,False,4.1,5.342334,0,...,False,False,False,False,False,False,True,False,False,False
5,0.320852,5,123,0.8,0.250000,0.812500,False,4.2,5.796058,2,...,False,False,False,False,False,True,False,False,False,False
6,1.439897,1,64,-2.7,-1.000000,0.900000,False,3.7,6.577861,1,...,False,False,True,False,False,False,False,False,False,False
7,0.580241,5,27,0.3,1.000000,0.300000,False,4.7,6.023448,0,...,False,False,False,False,False,False,True,False,False,False
8,-0.432168,1,137,-3.5,0.300000,0.300000,False,4.5,4.934474,0,...,False,False,False,False,False,False,True,False,False,False
9,0.802829,5,432,0.3,0.137500,0.617361,False,4.7,6.306275,2,...,False,False,False,False,False,False,True,False,False,False


is_outlier
 1    25571
-1     1051
Name: count, dtype: int64


'\nNumber of potential anomalies (z-score > 2.0): 1051'


Displaying the top 20 anomalies with all their features (sorted by anomaly score):
                     user_id   name_review_user           time  rating                                                                                                                                                   text                                gmap_id   latitude   longitude                                                                                                                                               category  avg_rating  num_of_reviews price                       state                   category_main  cat_American restaurant  cat_Coffee shop  cat_Department store  cat_Fast food restaurant  cat_Grocery store  cat_Hotel  cat_Mexican restaurant  cat_Other  cat_Pizza restaurant  cat_Restaurant  cat_Shopping mall  price_encoded  year  month  weekday  hour  text_len  log_num_reviews  rating_deviation  sentiment_polarity  sentiment_subjectivity  excessive_exclaim  ae_reconstruction_error

## Investigating the Latent Space

To better understand what the autoencoder has learned, we can inspect the latent space. 

Extract the encoded representation for each data point and create a new dataframe.

Then, calculate the correlation between these latent dimensions and the original features. This helped us interpret which original features are most strongly represented in the compressed dimensions of the latent space.

In [8]:
# Investigating the Latent Space
print("\n--- Investigating the Latent Space ---")

model.eval()
with torch.no_grad():
	encoded_data_tensor = model.encoder(scaled_data_tensor.to(device))
	encoded_data_np = encoded_data_tensor.cpu().numpy()

# Create a DataFrame for the encoded dimensions
encoded_df = pd.DataFrame(encoded_data_np, columns=[f'latent_dim_{i+1}' for i in range(encoding_dim)])

# Combine with the original (unscaled) features to make interpretation easier
combined_df = pd.concat([df[features].reset_index(drop=True), encoded_df], axis=1)

# Calculate the correlation matrix
correlation_matrix = combined_df.corr()

# Get the correlation of each latent dimension with the original features
latent_correlations = correlation_matrix[encoded_df.columns].loc[features]

print("\nCorrelation of Latent Dimensions with Original Features:")
print(latent_correlations)
print("\nStandard Deviation of each Latent Dimension:")
print(encoded_df.std())

df.to_parquet('AE_final_dataset_with_scores.parquet')
df.to_csv('AE_final_dataset_with_scores.csv', index=False)


--- Investigating the Latent Space ---

Correlation of Latent Dimensions with Original Features:
                          latent_dim_1  latent_dim_2  latent_dim_3  latent_dim_4  latent_dim_5  latent_dim_6  latent_dim_7  latent_dim_8  latent_dim_9  latent_dim_10
rating                        0.372923      0.187702     -0.297791      0.117751      0.224018     -0.179248      0.175195      0.002575     -0.047572      -0.198805
text_len                     -0.104157     -0.076832      0.115159     -0.081641     -0.041449      0.055121     -0.065347     -0.057584     -0.048256      -0.048435
rating_deviation              0.196693      0.269110     -0.264249      0.202052      0.055322     -0.049547      0.121217      0.073065     -0.012800      -0.199439
sentiment_polarity            0.351058      0.232133     -0.147438      0.093723      0.310283     -0.181689      0.284062      0.178351      0.075999       0.071297
sentiment_subjectivity        0.067063      0.266110      0.073971      

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26622 entries, 0 to 26621
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         26622 non-null  object 
 1   name_review_user                26622 non-null  object 
 2   time                            26622 non-null  int64  
 3   rating                          26622 non-null  int64  
 4   text                            26622 non-null  object 
 5   gmap_id                         26622 non-null  object 
 6   latitude                        26622 non-null  float64
 7   longitude                       26622 non-null  float64
 8   category                        26616 non-null  object 
 9   avg_rating                      26622 non-null  float64
 10  num_of_reviews                  26622 non-null  int64  
 11  price                           14747 non-null  object 
 12  state                           